In [1]:
import nibabel as nib
import sys
sys.path.append("..")
from config import *
import nibabel as nib
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import json
import pandas as pd
from utils import generate_BIDS_path, get_dataframe_from_split_lesions, load_lesions
import multiprocessing as mp

Loading configuration...
Configuration loaded successfully!
_____________________________



In [2]:
VERSION = "morpho_v02_maxDA"
MARGIN_SIZE = 11
DILATIONS_NUM = 5
VOLUME_FILTER = 45
DIM_FILTER = 24
MIN_PERCENTAGE_FOR_RIM_POS_PATCHES = 70
RANDOM_DA = 0. # old value: 0.4, the higher, the less DA there is. Value between 0 (maximum DA) and 1 (NO DA).

In [3]:
def create_folder_for_split_lesions(dataset_id, pat, der_folder, pipeline, session=1):
    # we create folder if it does not exist
    der_path = os.path.join(AVAILABLE_DATASETS_ROOTS[dataset_id], "derivatives", der_folder)
    if not os.path.exists(der_path):
        try:
            os.makedirs(der_path)
            print(f"[INFO] Derivatives folder for '{der_folder}' successfully created.")
        except: # Sometimes in multiprocessing this check is true for several processes and crashes
            pass
        
    # we create the description of the derivatives if it does not exist
    dataset_description_path = os.path.join(der_path, "dataset_description.json")
    if not os.path.exists(os.path.join(dataset_description_path)):
        descriptor = {
            "Name": der_folder,
            "BIDSVersion": BIDS_VERSION,
            "PipelineDescription": {
                "Name": pipeline,
                "version": VERSION,
                "margin_size": MARGIN_SIZE,
                "dilations_num": DILATIONS_NUM,
                "volume_filter": VOLUME_FILTER,
                "dim_filter": DIM_FILTER,
                "min_percentage_for_rim_pos_patches": MIN_PERCENTAGE_FOR_RIM_POS_PATCHES,
                "random_DA": RANDOM_DA,
            }
        }
        with open(dataset_description_path, "w") as outfile:
            json.dump(descriptor, outfile)
        print(f"[INFO] Description file for '{der_folder}' successfully created.")
    
    # we create the path for the generated file
    folder = os.path.join(der_path, f"sub-{pat:03d}", f"ses-{session:02d}")
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    return folder

In [4]:




def __extract_patch(mask):
    slice_x, slice_y, slice_z = ndimage.find_objects(mask)[0]
    roi = mask[slice_x, slice_y, slice_z]
    return roi, [slice_x.start, slice_y.start, slice_z.start]

def __get_and_remove_patches(original, lesions_guesses):
    centers = []
    result = original.copy()
    
    s = np.ones((3, 3, 3))
    labelled, num_objects = ndimage.measurements.label(lesions_guesses, structure=s)
    
    for l in range(1, num_objects+1):
        obj = (labelled == l).astype(int)
        com = [int(round(v)) for v in ndimage.measurements.center_of_mass(obj)]
        centers.append(com)
        
        ext = MARGIN_SIZE # SPHERE better??
        (x0, y0, z0) = com - np.array((ext, ext, ext))
        (x, y, z) = com + np.array((ext, ext, ext))
        lim = original.shape
        result[max(0, x0) : min(x, lim[0]), max(0, y0) : min(y, lim[1]), max(0, z0) : min(z, lim[2])] = 0
        
    return result, centers

def __paint_patch(mask, center, les_id, ext=MARGIN_SIZE):
    (x0, y0, z0) = center - np.array((ext, ext, ext))
    (x, y, z) = center + np.array((ext, ext, ext))
    lim = mask.shape
    mask[max(0, x0) : min(x, lim[0]), max(0, y0) : min(y, lim[1]), max(0, z0) : min(z, lim[2])] = les_id
    
def __get_centers_of_sublesions(one_lesion_mask):
    one_lesion_mask = one_lesion_mask.astype('int8')
    
    lesion_mask, pos0 = __extract_patch(one_lesion_mask)
    all_centers = []
    while(True):
        dt_mask = np.round(ndimage.morphology.distance_transform_edt(lesion_mask)).astype(int)
        max_val = np.max(dt_mask)
        if max_val <= 1:
            break
        aux = ndimage.morphology.binary_dilation((dt_mask >= max_val).astype(int), iterations=DILATIONS_NUM)
        
        lesion_mask, centers = __get_and_remove_patches(lesion_mask, aux)
        all_centers += [(c[0] + pos0[0], c[1] + pos0[1], c[2] + pos0[2]) for c in centers]
        """
        # check for small parts to remove
        labelled, num_objects = ndimage.measurements.label(lesion_mask)
        for l in range(1, num_objects+1):
            obj = (labelled == l).astype(int)
            vol = np.sum(obj)
            if vol <= VOLUME_FILTER:
                lesion_mask[obj.astype(bool)] = 0
                """
    
    all_centers.reverse()
    
    counter = 0
    patches = one_lesion_mask.astype(int)
    for center in all_centers:
        __paint_patch(patches, center, counter)
        counter += 1
    whole_mask = np.multiply(one_lesion_mask.astype(int), patches)
    return whole_mask, all_centers

def extract_lesions_from_segmentation(dataset_id, pat):
    dataset = AVAILABLE_DATASETS[dataset_id]
    path = dataset.get(return_type="filename", subject=f"{pat:03d}", **CONTRASTS["SEGMENTATION"])[0]
    mask_data = nib.load(path)
    mask = mask_data.get_fdata()
    _, n_labels_original = ndimage.measurements.label(mask)
    
    # prepare folder to save
    folder_name, pipeline = SPLIT_LESIONS_METADATA[VERSION]["folder_name"], SPLIT_LESIONS_METADATA[VERSION]["pipeline"]
    create_folder_for_split_lesions(dataset_id, pat, folder_name, pipeline)
    split_lesions_path = generate_BIDS_path(dataset_id, subject=f"{pat:03d}", scope=SPLIT_LESIONS_METADATA[VERSION]["pipeline"], suffix=SPLIT_LESIONS_METADATA[VERSION]["suffix"], acquisition=None, extension="nii.gz")
    
    # we start with the splitting
    counter_confluent = 0
    counter_whole = 0
    lesions = {}
    final_mask = np.zeros_like(mask)
    
    labelled, num_objects = ndimage.measurements.label(mask)
    for les in tqdm(range(1, num_objects+1)):
        '''
        1000 => whole
        2000 => splitted (if does not fit in a patch of 32x32x32)
        '''
        one_lesion_mask = (labelled == les).astype(int)
        vol = np.sum(one_lesion_mask)
        if vol < VOLUME_FILTER:
            # CLEANED because too small
            continue
        
        objects = ndimage.find_objects(one_lesion_mask)
        if(len(objects) != 1):
            print("ERROR")
        x, y, z = objects[0]
        biggest_dim = max(x.stop - x.start, y.stop - y.start, z.stop - z.start)
        if biggest_dim <= DIM_FILTER: # whole
            les = 1000 + counter_whole
            counter_whole += 1
            c = ndimage.measurements.center_of_mass(one_lesion_mask)
            lesions[les] = {"center": (int(round(c[0])), int(round(c[1])), int(round(c[2])))}
            final_mask[one_lesion_mask.astype(bool)] = int(les)
        else: # CONFLUENT lesion
            whole_mask, all_centers = __get_centers_of_sublesions(one_lesion_mask)
            #all_centers = __get_centers_of_sublesions(one_lesion_mask)
            whole_mask[whole_mask != 0] += 2000 + counter_confluent
            final_mask += whole_mask
            for center in all_centers:
                lesions[2000 + counter_confluent] = {"center": center}
                counter_confluent += 1
    
    
    nifti_out = nib.Nifti1Image(final_mask, mask_data.affine, mask_data.header)
    nifti_out.to_filename(split_lesions_path)
    return dataset_id, pat, n_labels_original, len(lesions.keys())
    #return final_mask, lesions

def retrieve_all_patients_centers(cpus = 6):
    pool = mp.Pool(min(cpus, mp.cpu_count()))
    processes = []
    results = []
    for dataset_id in range(len(AVAILABLE_DATASETS)):
        dataset = AVAILABLE_DATASETS[dataset_id]
        for pat in dataset.get_subjects():
            def callback(result):
                if result is None:
                    print("ERROR!")
                    return
                elif result == -1:
                    return
                d_id, pat, bef, after = result
                print(f"{d_id}.{pat} - TOTAL: {bef} -> {after} lesions.")
                results.append(result)
            processes.append(pool.apply_async(extract_lesions_from_segmentation, args=(dataset_id, int(pat)), callback=callback))

    for p in processes:
        p.get()
    pool.close()
    pool.join()


In [5]:
def match_split_lesions_with_GT(dataset_id, pat):
    np.random.seed(seed=0)
    
    #m = Munkres()
    
    folder_name, pipeline = SPLIT_LESIONS_METADATA[VERSION]["folder_name"], SPLIT_LESIONS_METADATA[VERSION]["pipeline"]
    
    # Loading of paths.
    dataset = AVAILABLE_DATASETS[dataset_id]
    split_lesions_path = generate_BIDS_path(dataset_id, subject=f"{pat:03d}", scope=SPLIT_LESIONS_METADATA[VERSION]["pipeline"], suffix=SPLIT_LESIONS_METADATA[VERSION]["suffix"], acquisition=None, extension="nii.gz")
    rimpos_annotations_paths = dataset.get(return_type="filename", subject=f"{pat:03d}", **CONTRASTS["EXPERTS_ANNOTATIONS"])
    meta_split_lesions_path = generate_BIDS_path(dataset_id, subject=f"{pat:03d}", scope=SPLIT_LESIONS_METADATA[VERSION]["pipeline"], suffix=SPLIT_LESIONS_METADATA[VERSION]["suffix"], acquisition=None, extension="csv")
    
    if not os.path.exists(split_lesions_path):
        print(f"Patient {dataset_id}.{pat} skipped because split lesions for version {VERSION} are missing.")
        return
    if os.path.exists(meta_split_lesions_path):
        print(f"Patient {dataset_id}.{pat} skipped because metadata of split lesions for version {VERSION} already exists.")
        return
    
    # IF there are annotations of rim+, we match them with the lesions.
    rimpos_centers = []
    split_lesions_centers = []
    
    
    # we retrieve them and make the association with split lesions
    m_image = nib.load(split_lesions_path)
    split_lesions = m_image.get_fdata()
    if len(rimpos_annotations_paths) > 0:
        gt_lesions = nib.load(rimpos_annotations_paths[0]).get_fdata()
    else:
        gt_lesions = np.zeros_like(split_lesions)
    o, h = m_image.affine, m_image.header

    labels_GT = np.unique(gt_lesions)
    labels_GT = np.delete(labels_GT, np.argwhere(labels_GT == 0))
    n_labels_GT = len(labels_GT)

    # quick check that everything is okay
    #assert n_labels_GT == len(df_lesions[(df_lesions["rim"]) & (df_lesions["dataset"] == dataset_id) & (df_lesions["patient"] == pat)])
    for lab in labels_GT:
        rimpos_centers.append([int(el) for el in ndimage.measurements.center_of_mass(gt_lesions == lab)])

    print(f"{dataset_id} - {pat} - {len(rimpos_centers)} rim+ lesion centers extracted.")

    labels_split = np.unique(split_lesions)
    labels_split = np.delete(labels_split, np.argwhere(labels_split == 0))
    n_labels_split = len(labels_split)
    for lab in labels_split:
        split_lesions_centers.append([int(el) for el in ndimage.measurements.center_of_mass(split_lesions == lab)])

    matrix_distances = np.zeros((len(rimpos_centers), len(split_lesions_centers)))
    matrix_intersection = np.zeros((len(rimpos_centers), len(split_lesions_centers)))
    matrix_intersection_perc = np.zeros((len(rimpos_centers), len(split_lesions_centers)))
    for i in range(len(rimpos_centers)):
        for j in range(len(split_lesions_centers)):
            label_GT, label_lesions = i+1, j+1
            # save the distance between them
            c1, c2 = rimpos_centers[i], split_lesions_centers[j]
            dist = np.linalg.norm(np.array(c1) - np.array(c2))
            matrix_distances[i,j] = dist
            # check for intersection
            if dist < 15: # to optimize
                current_les = (split_lesions == label_lesions).astype(int)
                current_rim_slice = (gt_lesions == label_GT).astype(int)
                matrix_intersection[i,j] = np.sum(current_les * current_rim_slice)
            else:
                matrix_intersection[i,j] = 0
                matrix_intersection_perc[i,j] = 0

    pos_patches = np.zeros_like(gt_lesions)
    result = np.zeros_like(gt_lesions)
    counter_rim = 1
    counter_non_rim = 1
    result_data = []
    MAX_REAL = len(split_lesions_centers)
    i = 0
    while i < len(split_lesions_centers):
        c = split_lesions_centers[i]
        # we build the bounding box and get the percentage of rim+ slice inside
        a = int(28/2) # PATCH SIZE = 28x28x28
        patch = gt_lesions[max(0, c[0]-a):min(c[0]+a, gt_lesions.shape[0]), 
                           max(0, c[1]-a):min(c[1]+a, gt_lesions.shape[1]), 
                           max(0, c[2]-a):min(c[2]+a, gt_lesions.shape[2])]
        perc_volume_rims_included_in_the_patch = [np.sum(patch == lab) / np.sum(gt_lesions == lab) for lab in np.unique(patch)[1:]]
        voxels_rims_included_in_the_patch = [np.sum(patch == lab) for lab in np.unique(patch)[1:]]

        maximum = 0 if len(perc_volume_rims_included_in_the_patch) == 0 else max(perc_volume_rims_included_in_the_patch)

        # FULL OF HYPERPARAMETERS:
        #     0.8 => we only want to augment those rim+ well fit inside the patch, so we can find patches around it that they still contain it.
        #     2 + np.random.rand() * 8 => distance to which generate the augmented patches.
        if i < MAX_REAL:# and maximum >= 0.8:
            # DA
            for x in (-1, 1):
                for y in (-1, 1):
                    for z in (-1, 1):
                        if np.random.rand() > RANDOM_DA:
                            split_lesions_centers.append((c[0] + x * int(np.ceil(2 + np.random.rand() * 8)), 
                                                          c[1] + y * int(np.ceil(2 + np.random.rand() * 8)), 
                                                          c[2] + z * int(np.ceil(2 + np.random.rand() * 8))))

        b = 2
        if maximum >= MIN_PERCENTAGE_FOR_RIM_POS_PATCHES / 100: # RIM+
            result_data.append((dataset_id, pat, 1000 + counter_rim, c[0], c[1], c[2], perc_volume_rims_included_in_the_patch, voxels_rims_included_in_the_patch, i < MAX_REAL))

            # for checking visually all lesions
            result[c[0]-b:c[0]+b, c[1]-b:c[1]+b, c[2]-b:c[2]+b] = 1000 + counter_rim

            # for checking visually the rim+ patches
            pos_patches[max(0, c[0]-a):min(c[0]+a, gt_lesions.shape[0]), 
                       max(0, c[1]-a):min(c[1]+a, gt_lesions.shape[1]), 
                       max(0, c[2]-a):min(c[2]+a, gt_lesions.shape[2])] = counter_rim
            a -= 1
            pos_patches[max(0, c[0]-a):min(c[0]+a, gt_lesions.shape[0]), 
                       max(0, c[1]-a):min(c[1]+a, gt_lesions.shape[1]), 
                       max(0, c[2]-a):min(c[2]+a, gt_lesions.shape[2])] = 0

            counter_rim += 1
        elif i < MAX_REAL: # RIM- => we do not risk to create rim- patches close to rim+ ones
            result_data.append((dataset_id, pat, 2000 + counter_non_rim, c[0], c[1], c[2], perc_volume_rims_included_in_the_patch, voxels_rims_included_in_the_patch, i < MAX_REAL))
            # for checking visually all lesions
            result[c[0]-b:c[0]+b, c[1]-b:c[1]+b, c[2]-b:c[2]+b] = 2000 + counter_rim
            counter_non_rim += 1
        i += 1

    nifti_out = nib.Nifti1Image(pos_patches, affine = o, header = h)
    nifti_out.to_filename(split_lesions_path.replace(".nii.gz", "_rimpos.nii.gz"))
    nifti_out = nib.Nifti1Image(result, affine = o, header = h)
    nifti_out.to_filename(split_lesions_path.replace(".nii.gz", "_result.nii.gz"))

    df = pd.DataFrame(result_data, columns=["dataset_id", "patient", "lesion", "x", "y", "z", "percentage_rims", "voxels_rims", "real"])
    df.to_csv(meta_split_lesions_path, index=False)
    return df


def match_all_patients_with_GT(cpus=6):
    data = []
    pool = mp.Pool(min(cpus, mp.cpu_count()))
    processes = []
    results = []
    for dataset_id in range(len(AVAILABLE_DATASETS)):
        dataset = AVAILABLE_DATASETS[dataset_id]
        for pat in dataset.get_subjects():
            def callback(result):
                if result is None:
                    #print(result)
                    return
                data.append(result)
            processes.append(pool.apply_async(match_split_lesions_with_GT, args=(dataset_id, int(pat)), callback=callback))

    for p in processes:
        p.get()
    pool.close()
    pool.join()

In [6]:
#retrieve_all_patients_centers()
#match_all_patients_with_GT()

In [4]:
from utils import load_lesions

lpp = load_lesions(PATCH_SIZE, from_segmentation=True, deformed=None, only_real=False, split_version=VERSION)

In [7]:
from utils import get_dataframe_from_metadata

df1 = get_dataframe_from_split_lesions(VERSION)
df2 = get_dataframe_from_metadata()

[WARNING] Subject 1 was omitted because no lesions 'json' file was found.
[WARNING] Subject 2 was omitted because no lesions 'json' file was found.
[WARNING] Subject 3 was omitted because no lesions 'json' file was found.
[WARNING] Subject 4 was omitted because no lesions 'json' file was found.
[WARNING] Subject 5 was omitted because no lesions 'json' file was found.
[WARNING] Subject 6 was omitted because no lesions 'json' file was found.
[WARNING] Subject 7 was omitted because no lesions 'json' file was found.
[WARNING] Subject 8 was omitted because no lesions 'json' file was found.
[WARNING] Subject 9 was omitted because no lesions 'json' file was found.
[WARNING] Subject 10 was omitted because no lesions 'json' file was found.
[WARNING] Subject 11 was omitted because no lesions 'json' file was found.
[WARNING] Subject 12 was omitted because no lesions 'json' file was found.
[WARNING] Subject 13 was omitted because no lesions 'json' file was found.
[WARNING] Subject 14 was omitted b

In [8]:
for (db, pat), grouped in df2.groupby(["dataset", "patient"]):
    rimpos_original = len(grouped[grouped["lesion"] // 1000 == 1].index)
    rimneg_original = len(grouped[grouped["lesion"] // 2000 == 1].index)
    df1_pat = df1[(df1["dataset_id"] == db) & (df1["patient"] == pat) & (df1["real"])]
    rimpos_now = len(df1_pat[df1_pat["lesion"] // 1000 == 1].index)
    rimneg_now = len(df1_pat[df1_pat["lesion"] // 2000 == 1].index)
    print(f"[{db}-{pat}: + {rimpos_original} -> {rimpos_now}, - {rimneg_original} -> {rimneg_now}]")

[0-56: + 0 -> 0, - 8 -> 8]
[0-57: + 3 -> 3, - 2 -> 2]
[0-58: + 1 -> 1, - 31 -> 26]
[0-59: + 0 -> 0, - 10 -> 6]
[0-60: + 1 -> 3, - 34 -> 36]
[0-61: + 0 -> 0, - 115 -> 59]
[0-62: + 0 -> 0, - 8 -> 7]
[0-63: + 0 -> 0, - 28 -> 25]
[0-64: + 11 -> 22, - 239 -> 196]
[0-65: + 3 -> 3, - 140 -> 94]
[0-66: + 4 -> 4, - 10 -> 7]
[0-67: + 2 -> 2, - 16 -> 16]
[0-68: + 0 -> 0, - 52 -> 31]
[0-69: + 1 -> 1, - 1 -> 1]
[0-70: + 6 -> 5, - 40 -> 30]
[0-71: + 0 -> 0, - 69 -> 56]
[0-72: + 3 -> 3, - 40 -> 22]
[0-73: + 0 -> 0, - 53 -> 27]
[0-74: + 6 -> 10, - 53 -> 36]
[0-75: + 5 -> 4, - 51 -> 39]
[0-76: + 0 -> 0, - 74 -> 49]
[0-77: + 2 -> 3, - 24 -> 18]
[0-78: + 8 -> 8, - 55 -> 34]
[0-79: + 1 -> 1, - 91 -> 79]
[0-80: + 1 -> 1, - 16 -> 12]
[0-81: + 4 -> 3, - 74 -> 52]
[0-82: + 0 -> 0, - 32 -> 17]
[0-83: + 1 -> 1, - 81 -> 49]
[0-84: + 18 -> 26, - 111 -> 59]
[0-85: + 32 -> 43, - 99 -> 60]
[0-92: + 9 -> 14, - 71 -> 45]
[0-93: + 28 -> 31, - 145 -> 105]
[0-94: + 9 -> 13, - 54 -> 34]
[0-95: + 16 -> 28, - 102 -> 113]
[0

In [9]:
df1.loc[(df1["lesion"] // 2000 == 1) & (df1["dataset_id"] != 2)]

,dataset_id,patient,lesion,x,y,z,percentage_rims,voxels_rims,real
0,0,56,2001,98,129,262,[],[],True
1,0,56,2002,97,238,263,[],[],True
2,0,56,2003,159,215,271,[],[],True
3,0,56,2004,162,222,266,[],[],True
4,0,56,2005,164,250,245,[],[],True
...,...,...,...,...,...,...,...,...,...
13,1,55,2014,159,262,218,[],[],True
15,1,55,2015,103,161,263,[],[],True
16,1,55,2016,146,145,239,[],[],True
17,1,55,2017,150,145,253,[],[],True


In [12]:
df1.loc[(df1["lesion"] // 1000 == 1) & (df1["dataset_id"] != 2)]

,dataset_id,patient,lesion,x,y,z,percentage_rims,voxels_rims,real
1,0,57,1001,89,251,238,[1.0],[655],True
2,0,57,1002,100,219,260,[1.0],[298],True
4,0,57,1003,106,130,277,[0.9867330016583747],[1190],True
5,0,57,1004,92,228,254,[0.9865771812080537],[294],False
6,0,57,1005,82,248,231,[1.0],[655],False
...,...,...,...,...,...,...,...,...,...
21,1,55,1004,95,224,273,[1.0],[135],False
22,1,55,1005,105,213,260,[1.0],[135],False
23,1,55,1006,105,212,275,[0.9111111111111111],[123],False
24,1,55,1007,110,229,256,[1.0],[135],False


In [ ]:

lpp = load_lesions(PATCH_SIZE, from_segmentation=True, deformed=None, only_real=False, split_version=VERSION)

[START] Loading lesions (deformed=None, asyncr=True, cpus=8, segm=True)...


Process ForkPoolWorker-10:
Process ForkPoolWorker-16:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
